In [25]:
#!pip -q install langchain openai tiktoken

In [22]:
import os
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [45]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

## Wikipedia

In [46]:
#!pip -q install wikipedia

In [47]:
from langchain.utilities import WikipediaAPIWrapper

In [48]:
wikipedia = WikipediaAPIWrapper()

In [49]:
wikipedia.run('Langchain')

"Page: LangChain\nSummary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\n\n\nPage: Sentence embedding\nSummary: In natural language processing, a sentence embedding refers to a numeric representation of a sentence in the form of a vectors of real numbers which encodes meaningful semantic information.State of the art embeddings are based on the learned hidden layer representation of dedicated sentence transformer models. BERT pioneered an approach involving the use of a dedicated [CLS] token preprended to the beginning of each sentence inputted into the model; the final hidden state vector of this token encodes information about the sentence and can be fine-tuned for use in sentence classification tasks. In practice however,

## Python REPL

In [50]:
from langchain.utilities import PythonREPL

In [51]:
python_repl = PythonREPL()

In [52]:
python_repl.run("print(17*2)")

'34\n'

## DuckDuckGo

In [58]:
!pip -q install duckduckgo-search

In [78]:
query = "who whon the 1990 world cup"

In [79]:
from itertools import islice
from duckduckgo_search import DDGS

class DDG(DDGS):
    def run (query:str):
        '''
        "snippet": result["body"],
        "title": result["title"],
        "link": result["href"],
        '''
        results = DDGS().text(query)
        generator = [result['body'] for result in results]

        # Get the first three results from the generator
        first_three_results = list(islice(generator, 3))
        return first_three_results

In [95]:
# from langchain import OpenAI
# llm = OpenAI(temperature=0)

# This is way cheaper! 
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", max_tokens=1000)

## Math chain

In [104]:
from langchain import LLMMathChain

#llm_math_chain = LLMMathChain(llm=llm, verbose=True) <- Deprecated !!!
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
math_tool = Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions.",
    )

In [105]:
from langchain.agents import Tool

tools = [
    Tool(
        name = "python repl",
        func=python_repl.run,
        description="useful for when you need to use python to answer a question. You should input python code"
    ),
    Tool(
        name='wikipedia',
        func= wikipedia.run,
        description="Useful for when you need to look up a topic, country or person on wikipedia"
    ),
    Tool(
        name='DuckDuckGo Search',
        func= DDG.run,
        description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)]

In [106]:
tools.append(math_tool) # example of adding a tool retrospectively:

In [107]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
)

In [108]:
query_1="If I have 28 apples, and eat 2, and then \
    get given 4, please can you calculate 10% of the total number of apples left?"

In [109]:
with get_openai_callback() as cb:
    response = zero_shot_agent.run(query_1)
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")



> Entering new  chain...
I need to calculate the total number of apples left after eating and receiving some, and then find 10% of that number.
Action: Calculator
Action Input: (28 - 2 + 4) * 0.1

> Entering new  chain...
(28 - 2 + 4) * 0.1```text
(28 - 2 + 4) * 0.1
```
...numexpr.evaluate("(28 - 2 + 4) * 0.1")...

Answer: 3.0
> Finished chain.

Observation: Answer: 3.0
Thought:I now know the final answer
Final Answer: 3.0

> Finished chain.
Total Tokens: 927
Prompt Tokens: 826
Completion Tokens: 101
Total Cost (USD): $0.001441
